In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
from tqdm import tqdm

%matplotlib inline

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [10]:
nclasses = 10
batchSize = 128
keepRate = 0.8
epochs = 25

In [11]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [12]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [13]:
def cnn(x):
    weights = {
        'Wconv1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
        'Wconv2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
        'Wfc1': tf.Variable(tf.random_normal([7*7*64, 1024])),
        'out': tf.Variable(tf.random_normal([1024, nclasses]))
    }
    
    biases = {
        'bconv1': tf.Variable(tf.random_normal([32])),
        'bconv2': tf.Variable(tf.random_normal([64])),
        'bfc1': tf.Variable(tf.random_normal([1024])),
        'out': tf.Variable(tf.random_normal([nclasses]))
    }
    
    x = tf.reshape(x, [-1, 28, 28, 1])
    
    out = tf.nn.relu(conv2d(x, weights['Wconv1']) + biases['bconv1'])
    out = maxpool2d(out)
    
    out = tf.nn.relu(conv2d(out, weights['Wconv2']) + biases['bconv2'])
    out = maxpool2d(out)
    
    out = tf.reshape(out, [-1, 7*7*64])
    
    out = tf.nn.relu(tf.matmul(out, weights['Wfc1']) + biases['bfc1'])
    out = tf.nn.dropout(out, keepRate)
    
    out = tf.matmul(out, weights['out']) + biases['out']
    
    return out

In [14]:
def trainCNN(x):
    losses = []
    prediction = cnn(x)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            epochLoss = 0

            for _ in range(int(mnist.train.num_examples/batchSize)):
                batchX, batchY = mnist.train.next_batch(batchSize)
                _, l = sess.run([optimizer, loss], feed_dict={x: batchX, y: batchY})
                epochLoss += l
            
            losses.append(epochLoss)
            print('Epoch', epoch+1, '/', epochs, 'loss:', epochLoss)
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        acc = tf.reduce_mean(tf.cast(correct, tf.float32))
        print('Accuracy: ', sess.run([acc], feed_dict={x: mnist.test.images, y: mnist.test.labels}))
    
    return losses

In [ ]:
l = trainCNN(x)

Epoch 1 / 25 loss: 2395878.2810668945
Epoch 2 / 25 loss: 317370.2339324951
Epoch 3 / 25 loss: 165726.50879478455
Epoch 4 / 25 loss: 105114.65452575684
Epoch 5 / 25 loss: 71654.57000350952
Epoch 6 / 25 loss: 51605.71741902828
Epoch 7 / 25 loss: 40811.461614608765
Epoch 8 / 25 loss: 29173.414976119995
Epoch 9 / 25 loss: 22923.841498318594
Epoch 10 / 25 loss: 18500.5335360373
Epoch 11 / 25 loss: 14265.773971661925
Epoch 12 / 25 loss: 13089.773144918494
Epoch 13 / 25 loss: 9616.946949005127
Epoch 14 / 25 loss: 9838.689072646437
Epoch 15 / 25 loss: 7551.2342908466235
Epoch 16 / 25 loss: 7532.2962846304945


In [ ]:
plt.plot(range(epochs), l)
plt.xlabel('Epoch')
plt.ylabel('Loss')